This notebook contains all relevant code regarding text explanatory data analysis (EDA), audio to text transcription (German File)  using different models. Comments about the models are found at the end.

Table of contents:

1. EDA (using AWS Transcription German 'ukush_2' document & 'pastuer' document)
2. Audio Models
  - Whisper
    - Tiny
    - Base
    - Medium
    - Large
  - Vosk
  - Assembly AI
  - Salad Transcription

For evaluation of the German Medical Dataset and ground truth, refer to 'Dataset Evaluation" notebook.

# Exploratory Data Analysis

In [ ]:
! pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Load Packages
import json
import re
import yake
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import wave
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment
from collections import Counter
import whisper
import os
import time

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Define Functions

# Text preprocessing ( remove stop words, punctuation, numbers, lowercase
def preprocess_text(text, language):
    text = text.lower()  # Lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers + punctuation in order to just analyze words ( we do not find it valuable to analyze the time stamps )
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    stop_words = set(stopwords.words(language))  # Stopwords in German
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def analyze_text(clean_text, language="english"):
    # Basic Analysis
    word_count = len(clean_text.split())
    word_frequencies = Counter(clean_text.split())
    most_common_words = word_frequencies.most_common(10)

    # Keyword Extraction using TF-IDF
    vectorizer = TfidfVectorizer(max_features=10)
    tfidf_matrix = vectorizer.fit_transform([clean_text])
    tfidf_keywords = vectorizer.get_feature_names_out()

    # Keyword Extraction using YAKE
    yake_extractor = yake.KeywordExtractor(lan="de" if language == "german" else "en", n=2, top=10)
    yake_keywords = [kw[0] for kw in yake_extractor.extract_keywords(clean_text)]

    # Check common keywords between TF-IDF & YAKE
    common_keywords = set(tfidf_keywords) & set(yake_keywords)

    return {
        "Word Count": word_count,
        "Most Common Words": most_common_words,
        "TF-IDF Keywords": list(tfidf_keywords),
        "YAKE Keywords": yake_keywords,
        "Common Keywords": list(common_keywords)
    }

Load Documents

In [ ]:
# Load Pasteur
file_path_pasteur = "/kaggle/input/pastuer-tmvr/Pasteur_TMVR.txt"

with open(file_path_pasteur, "r", encoding="utf-8") as f:
    pasteur_text = json.load(f)

# Load German text
file_path_german = "/kaggle/input/german-text/uksh_2.json"

with open(file_path_german, "r", encoding="utf-8") as f:
    german_text = json.load(f)

In [ ]:
# clean documents
pasteur_text = " ".join([entry["transcript"] for entry in pasteur_text])
pasteur_clean = preprocess_text(pasteur_text, "english")

german_text = " ".join([entry["transcript"] for entry in german_text["results"]["transcripts"]])
german_clean = preprocess_text(german_text, "german")

## Frequency & Key Word Analysis:

In [ ]:
results = analyze_text(german_clean, language="german")
print(results)


{'Word Count': 3060, 'Most Common Words': [('ja', 288), ('mal', 102), ('bisschen', 67), ('schon', 47), ('ne', 37), ('gut', 36), ('schön', 26), ('genau', 23), ('zurück', 23), ('ganz', 22)], 'TF-IDF Keywords': ['bisschen', 'ganz', 'genau', 'gut', 'ja', 'mal', 'ne', 'schon', 'schön', 'zurück'], 'YAKE Keywords': ['mal bisschen', 'mal mal', 'guck mal', 'schau mal', 'zieh mal', 'mal', 'lass mal', 'sag mal', 'okay mal', 'warte mal'], 'Common Keywords': ['mal']}


In [ ]:
results_eng = analyze_text(pasteur_clean, language="english")
print(results_eng)

{'Word Count': 1837, 'Most Common Words': [('uh', 188), ('see', 36), ('yeah', 36), ('ok', 33), ('valve', 29), ('um', 27), ('position', 22), ('think', 20), ('mitral', 17), ('maybe', 17)], 'TF-IDF Keywords': ['fusion', 'maybe', 'ok', 'position', 'see', 'think', 'uh', 'um', 'valve', 'yeah'], 'YAKE Keywords': ['mitral valve', 'stiff wire', 'yeah yeah', 'good position', 'valve valve', 'fusion imaging', 'position valve', 'septum yeah', 'wire position', 'left ventricle'], 'Common Keywords': []}


Comments: Can see that the most common words are filler or ocnversational words. YAKE did a good job at extracting medical vocabulary for the english text ( which is signifcantly less conversational than the German one). However, the German YAKE key words include non medical and more conversational words. Here are the terms translated ('a little bit', 'a little time', 'look', 'look', 'pull', 'let's', 'tell me', 'okay times', 'wait a minute'). We can see the difference a more structered operation makes in comparison to one with signifcantly more conversation.

# Calling Audio Transcription Models

## Whisper OpenAi

OpenAI Whisper is an advanced automatic speech recognition (ASR) system designed to transcribe and translate spoken language. It is available in different versions: tiny, base, small, medium, and large. The tiny model is the smallest and fastest among them, optimized for efficiency in real-time applications and low-resource environments. While it offers quick processing speeds, it has lower accuracy compared to larger models, especially when dealing with complex accents, noisy backgrounds, or long-form speech. Despite these limitations, the tiny model remains a powerful tool for basic speech recognition tasks where performance and speed are the primary concerns.


In [ ]:
! pip install openai--whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 7.2 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai--whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803403 sha256=819746be6b537fa57642d2f2b8d76ac7894594d3878c1fe855f8a225d6493ec4
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai--whisper


### Tiny Model

In [ ]:
tiny_model = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 64.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# load tiny model
start_time = time.time()

result_tiny_whisper = tiny_model.transcribe(germ_audio)

# End timer
end_time = time.time()

# Calculate and print elapsed time
elapsed_time = end_time - start_time
print(f"Transcription completed in {elapsed_time:.2f} seconds.")

Transcription completed in 162.53 seconds.


The latency is not bad.

In [ ]:
# result_tiny_whisper

### Base Model

In [ ]:
base_model = whisper.load_model("base")

start_time = time.time()

result_base_whisper = base_model.transcribe(germ_audio)

# End timer
end_time = time.time()

# Calculate and print elapsed time
elapsed_time = end_time - start_time
print(f"Transcription completed in {elapsed_time:.2f} seconds.")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 92.2MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription completed in 207.29 seconds.


Base model took ~40 seconds longer to run.


In [ ]:
result_base_whisper

{'text': " Test Test record. Wir sind in der Vorbereitung der Prozedur. Sorry, ich sage nicht, findet sie noch nicht. Also an die KI, wir sind noch nicht warm Tisch. Video funktioniert leider nicht. Hoffentlich funktioniert der Ton. Ich habe mal meine heutigen Handys hier hingelegt, ja? Die vier Stück. Und dann bist du schon im Weinern. Habt ihr auch diese Kita App-Familie App bei euch in der Kita? Ja, ja. Ja, es praktisch, schnell. Das finde ich auch. Family. Ja, ja, ja, und dann guckst du rein. Wir sind cool mit Apfel, Chili und Croquetten. Das haben wir beeindruckend. Wenn sie auch so lecker in der Kita heute, dann guckst du rein. Es gab Brokoli mit ... Ja, ich kann uns doch aussehen. Ja, Mama, der Kartoffelbrein der Kita, der ist so gut, dann machst du einen Verscheißen. Ein. Ja,❤❤❤❤km❤demo, CAN Sch compatopodener. Von Zach. Das ist Sorry. Ich finde das ist richtig geil. Danke. Das ist meine Ernst. Der braucht viel. Da i noch zumatted vorbei Ja. Aber Eike, es muss mir einer von euc

### Medium Model

In [ ]:
medium_model = whisper.load_model("medium")

start_time = time.time()

result_medium_whisper = medium_model.transcribe(germ_audio)

# End timer
end_time = time.time()

# Calculate and print elapsed time
elapsed_time = end_time - start_time
print(f"Transcription completed in {elapsed_time:.2f} seconds.")

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 130MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription completed in 321.91 seconds.


In [ ]:
result_medium_whisper

{'text': " Test, Test, Test. Record. Wir sind in der Vorbereitung der Prozedur. Sorry, ich sag nicht, findet sie noch nicht. Also an die KI, wir sind noch nicht mal am Tisch. Video funktioniert leider nicht. Hoffentlich funktioniert der Ton. Ich hab mal meine heutigen Handys hier hingelegt, die vier Stück. Na, bist du schon im weiteren? Habt ihr auch diese Kita-App, Family-App bei euch in der Kita? Ja, wir auch. Ja, ist praktisch. Find ich auch. Family. Ja, ja, ja, und dann guckst du rein. Würsingkohl mit Apfelgelee und Kroketten. Ist auch mal beeindruckend. Es war so lecker in der Kita heute. Und dann guckst du rein, es gab Brokkoli mit ja, zu Hause. Dann, ja, Mama, der Kartoffelbrei in der Kita, der ist so gut, dann machst du zu Hause Kartoffelbrei. Und dann ist es so, dass du auch die beste Köchin bist. Und ich denke mir, okay, es ist ein Appetito essen, was die einfach nur machen. Ja, ja. Alles klar. Das ist so fies. Das war sorry, ey. Ich finde richtig geil. Danke. Nein, dass ihr 

### Large Model

In [ ]:
large_model = whisper.load_model("large")

start_time = time.time()

result_large_whisper = large_model.transcribe(germ_audio)

# End timer
end_time = time.time()

# Calculate and print elapsed time
elapsed_time = end_time - start_time
print(f"Transcription completed in {elapsed_time:.2f} seconds.")

100%|█████████████████████████████████████| 2.88G/2.88G [00:34<00:00, 89.7MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription completed in 1646.78 seconds.


In [ ]:
result_large_whisper

{'text': ' Test, Test. Rekord. Wir sind in der Vorbereitung der Prozedur. Sorry, findet sie noch nicht. Also an die KI, wir sind noch nicht mal am Tisch. Video funktioniert leider nicht. Hoffentlich funktioniert der Ton. Ich habe mal meine heutigen Handys hier hingelegt, ja? Die vier Stück. Na, bist du schon im Weihnachten? Habt ihr auch diese Kita-App, Family-App bei euch in der Kita? Ja, ist praktisch, ne? Das finde ich auch. Family. Ja. Ja, ja, ja, und dann guckst du. Dann guckst du rein. Wirsingkohl mit Apfelgelee und Kroketten. Das war beeindruckend. Ach, es war so lecker in der Kita heute. Und dann guckst du rein. Es gab Brokkoli mit ja, und zu Hause dann, ja, Mama, der Kartoffelbrei in der Kita, der ist so gut, dann machst du zu Hause Kartoffelbrei. Ja, ich denke mir, Mama, Andrea ist die beste Köchin. Und ich denke mir, okay, es gibt Appetito-Essen, was die einfach nur macht. Ja, ja. Alles klar. So fies. Sorry, ey. Ich finde das richtig geil. Danke. Danke. Dass ihr das so sagt.

In [ ]:
!pip install openai pydub

## Vosk's ASR Model

In [ ]:
!pip install vosk pydub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 42.4 MB/s eta 0:00:0000:0100:01
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=f710ba34e0b719fc4a2b20624797dd9337fd00d54be2b8480842c948fa4dc7a8
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt


In [ ]:
model_path = "/kaggle/input/german-vosk-model/vosk-model-de-0.21"

try:
    model = Model(model_path)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")


Model loaded successfully!


In [ ]:
model_path = "/kaggle/input/german-vosk-model/vosk-model-de-0.21"
model = Model(model_path)

# Convert MP3 to WAV -- Vosk requires .wav format
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export(wav_path, format="wav")

# Transcribe Audio with Vosk
def transcribe_audio(audio_path, model_path= "/kaggle/input/german-vosk-model/vosk-model-de-0.21"):
    model = Model(model_path)

    wf = wave.open(audio_path, "rb")
    recognizer = KaldiRecognizer(model, wf.getframerate())

    # Process audio
    transcription = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            transcription.append(result.get("text", ""))

    # Get final transcription
    final_result = json.loads(recognizer.FinalResult())
    transcription.append(final_result.get("text", ""))

    return " ".join(transcription)


In [ ]:
start_time = time.time()
wav_file = "german_audio.wav"

convert_mp3_to_wav(germ_audio, wav_file)  # Convert MP3 to WAV
transcribed_text = transcribe_audio("/kaggle/working/german_audio.wav")  # Transcribe WAV

end_time = time.time()

print("Transcription:", transcribed_text)

# Calculate and print elapsed time
elapsed_time = end_time - start_time
print(f"Transcription completed in {elapsed_time:.2f} seconds.")

Transcription: test test record wir sind in der vorbereitung der prozedur  story die sonne findet sie noch nicht allein die ki wir sind noch nicht wandtisch video funktioniert leider nicht hoffentlich funktioniert der ton hm  ich hab mal meine meine heutigen handys hier hingelegt ja die vier stück na wisst ihr schon im weiteren habt ihr auch diese kita app family app bei euch in der kita er ja er ist praktisch schnell wmf finde ich auch famlie ja ja ja ja und dann guckst du rein wirsingkohl mit apfel gelee und kroketten sauna beeindrucken in der alpha so lecker in der in der kita heute und guckst du rein es gab brokkoli mit ja und zu hause dann ja mama der kartoffelbrei der kita der so gut masse sollte da doch was peaches ja ja sophies   das fest sorry das ist ich finde dich geil danke wenn das ihr das sagte weil es meine ernst  der braucht viel   hm  die fahr ich schon so bisschen ich wollte gar schon mal so bisse aus dem weg fahren tatsächlich so       können aber eike es muss mir ei

## Assembly Ai

In [ ]:
! pip3 install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
import assemblyai as aai

aai.settings.api_key = "ENTER API KEY HERE"

config = aai.TranscriptionConfig(language_code="de", speech_model= aai.SpeechModel.best, content_safety=True)

transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe("https://drive.google.com/uc?export=download&id=1IhpGNmTGbn76Jn1ZyEZApZt2M3MxgppR")

# print(transcript.text)

for result in transcript.content_safety.results:
    print(result.text)
    print(f"Timestamp: {result.timestamp.start} - {result.timestamp.end}")
    # Get category, confidence, and severity.
    for label in result.labels:
      print(f"{label.label} - {label.confidence} - {label.severity}")  # content safety category
    print()


Die sind zusätzlich gekommen, ungeplant, weil wir haben 996 Ablationen, aber wir wollen 1000. Und deswegen mussten wir noch vier unterkriegen. Zwei kommen heute mit an den Tisch. Ja, wenn du da bist, klar. Ja, also. Und Montag kommen auch noch mal zwei. Nein, nein, nein. Montag zur Aufklärung eine Ablation. Zwei Schrittmacher sind Montag geplant. Genau. Doch, eine Ablation kommt da noch hin. Ne, ne, alle machen wir als PFA, damit es schnell geht.
Timestamp: 484690 - 523922
ContentSafetyLabel.health_issues - 0.75155807 - 0.34563136

Alle als Fahrerpass. Wir sind bei 996 und wir müssen daraus eine vierstellige Zahl machen. Ja, wir haben jetzt am Montag eine Ablation und dann zwischen den paar Tagen noch zwei Schrittmacher und eine Ablation. Und wir haben nur junge, unkomplizierte Patienten, die eine First du mit PFA kriegen, geplant. Also unsere schnell eine am Montag und eine drei zwischen den Jahren noch. Irgendwas ist mit diesen Beinen komisch. Der hat die Füße überkreuzt.
Timestamp: 

In [ ]:
print(transcript.text)

Test, Test Record. Wir sind in der Vorbereitung der Prozedur. Jetzt findet die Transetate statt. Sorry, findet sie noch nicht. Also an die KI. Wir sind noch nicht mal am Tisch. Video funktioniert leider nicht. Hoffentlich funktioniert der Ton. Ich habe mal meine. Meine heutigen Handys hier hingelegt. Ja. Die vier Stück. Na, bist du schon im Weihnachten? Habt ihr auch diese Kita App, Family App bei euch in der Kita? Ja. Wir auch. Ja, ist praktisch. Das finde ich auch. Family. Ja. Und dann guckst du rein. Wirsingkohl mit Apfelgelee und Kroketten. Ist auch mal beeindruckend, wenn sie. Es war so lecker in der Kita heute. Und dann guckst du rein. Es gab Brokkoli mit. Ja. Und zu Hause dann. Ja. Mama, der Kartoffelbrei in der Kita, der ist so gut. Dann machst du zu Hause Kartoffelbrei. Es gibt Appetito. Essen, was die einfach. Ja, das. Sorry. Ey, ich finde richtig geil. Danke, dass ihr das so sagt. Das meine ich ernst. Der braucht viel. Die fahre ich schon so ein bisschen. Ich wollte die gera

## Salad Transcription

In [ ]:
import os
import requests
import json
import base64


def send_transcription_request(audio_url, organization_name, salad_key, language_code="en", input_params=None):
    """
    Sends a transcription request to the Salad Transcription API with dynamic input parameters.

    Args:
        audio_url (str): The signed Salad Storage URL for the audio file.
        api_endpoint (str): The Salad Transcription API endpoint.
        salad_key (str): The Salad API key for authentication.
        language_code (str): The language of the audio file (default: "en").
        input_params (dict): Additional input parameters to include in the request.

    Returns:
        dict: The response from the API, including the job ID.
    """
    # Set headers
    headers = {
        "Salad-Api-Key": salad_key,
        "Content-Type": "application/json",
        "accept": "application/json",
    }
    api_endpoint = f"https://api.salad.com/api/public/organizations/{organization_name}/inference-endpoints/transcribe/jobs"
    # Prepare the input dictionary, appending the audio URL and language code
    input_data = input_params.copy() if input_params else {}
    input_data["url"] = audio_url
    input_data["language_code"] = language_code

    # Request body
    data = {
        "input": input_data
    }

    # Send the request
    response = requests.post(api_endpoint, headers=headers, json=data)
    response.raise_for_status()  # Raise an error for bad responses

    # Extract and return the response
    return response.json()

In [ ]:
def get_transcription_result(job_id, salad_key, organization_name):
    """
    Retrieves the transcription result from the Salad Transcription API for a given job ID.

    Args:
        job_id (str): The unique identifier of the transcription job.
        salad_key (str): The Salad API key for authentication.
        organization_name (str): Name of the Salad organization.

    Returns:
        dict: The response from the API containing the job status and transcription result.
    """
    # Construct the Salad API endpoint for retrieving job results
    api_endpoint = f"https://api.salad.com/api/public/organizations/{organization_name}/inference-endpoints/transcribe/jobs/{job_id}"

    # Set headers
    headers = {
        "Salad-Api-Key": salad_key,
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    # Send the request
    response = requests.get(api_endpoint, headers=headers)
    response.raise_for_status()

    # Extract and return the response
    return response.json()

In [ ]:
salad_response = " Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise 12 Stunden bis 3 Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise 9 Tage, bei Kindern bis zu 15 Tage. Erhöhte Cholesterinwerte im Blut, auch Ichpercholesterinämie genannt, werden vom Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erbgut erhöhte Blutfettwerte aus, dann oft schon im Kindesalter. Die Aufgabe des LDL ist es, Cholesterin aus der Leber zu den Körperzellen zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus. Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokopen. Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Influenzaviren ansteckt, befallen sie die Atemwege und zerstören dort die Zellen der Schleimhaut. Die Arthrose ist eine schmerzhafte Gelenkerkrankung. Wenn man eine Grippeerkrankung überstanden hat, ist der Körper immun gegen das Influenzavirus, das die Erkrankung ausgelöst hat. Eine Lungenentzündung, auch Pneumonie genannt, ist eine plötzlich auftretende Erkrankung der unteren Atemwege. An Grippe oder Influenza erkranken vor allem in der kalten Jahreszeit vermehrt Menschen. Grippe beginnt typischerweise plötzlich mit Fieber, trockenem Husten, Halsschmerzen, Muskelschmerzen und Kopfschmerzen. Schätzungsweise zwischen 5 und 20% der Bevölkerung stecken sich jährlich in Deutschland mit Influenzaviren an. Wenn die Lunge entzündet ist, dann kann sich auch die dünne Haut um die Lunge herum das Brustfell entzünden. Unter Adipositas versteht man ein zu hohes Körpergewicht, das durch einen übermäßig hohen Fettanteil verursacht wird. Der hohe Fettanteil entsteht, wenn dem Körper, etwa durch eine kalorienreiche Ernährung, mehr Energie zugefügt wird. Körperliche Aktivität reduziert das Risiko für Herzerkrankungen, Schlaganfälle und Diabetes. Enzephalitis ist eine Entzündung des Gehirns, die durch Viren oder andere Erreger verursacht wird. Arthrose entsteht, wenn der Gelenkknorpel irreparabel geschädigt wird. Meningokokken sind Bakterien, die eine schwere Infektion der Hirnhäute oder eine Blutvergiftung verursachen können. Stress spielt bei der Entstehung einer Depression eine entscheidende Rolle. Schwere und chronische Krankheiten sind zudem eine Dauerbelastung für die Psyche. Diabetes mellitus ist eine krankhafte Störung des Zuckerstoffwechsels. Körperliche Gesundheit bedeutet, dass der körperliche Zustand eines Menschen keinen negativen Einflüssen, Einschränkungen oder Verletzungen unterliegt. Psychische Gesundheit ist ein Zustand, in dem ein Mensch keinen seelischen und geistigen Einschränkungen oder Störungen unterliegt. Eine Erkältung oder auch grippaler Infekt ist eine Erkrankung der oberen Atemwege."

In [ ]:
salad_response

' Manche Krankheiten treten gehäuft zu bestimmten Jahreszeiten auf. Zwischen der Ansteckung und dem Krankheitsbeginn liegen üblicherweise 12 Stunden bis 3 Tage. Eine Erkältung beginnt in vielen Fällen zunächst mit Halsschmerzen. Eine unkomplizierte Erkältung dauert normalerweise 9 Tage, bei Kindern bis zu 15 Tage. Erhöhte Cholesterinwerte im Blut, auch Ichpercholesterinämie genannt, werden vom Betroffenen oft längere Zeit nicht bemerkt. In seltenen Fällen löst ein spezifischer Defekt im Erbgut erhöhte Blutfettwerte aus, dann oft schon im Kindesalter. Die Aufgabe des LDL ist es, Cholesterin aus der Leber zu den Körperzellen zu bringen. Wenn eine Lungenentzündung frühzeitig festgestellt wird, ist sie aber meist gut behandelbar und heilt in der Regel folgenlos aus. Lungenentzündungen werden meist durch Bakterien ausgelöst, insbesondere durch Pneumokopen. Pilze können vor allem bei Menschen mit starker Immunschwäche eine Lungenentzündung hervorrufen. Wenn sich eine Person mit Influenzavire

**Model Comments**

*Whisper: *
We can see clear differences between the tiny, base, medium and large models. Some clear distinctions between them are as follows:

1. In the base model, we see that the audio was not transcribed properly resulting in this:  Ja,❤❤❤❤km❤demo. This is not present in any other model
2. We can see differences in the number of consecutive " Ja Ja Ja". Sometimes there are just a few and soemtimes there are 10-20 in same position but ti varies model to model
3. The medium model seems to either not understand the audio or is pporly transcribing it. At the end of the text block "Die Hinterwand ist schön.  Die Abteilung ist in der Abteilung. Die Abteilung ist in der Abteilung. Die Abteilung ist in der Abteilung. Die Abteilung ist in der Abteilung." is written hundres of times
4. Large model seems to give best results meaning no repetitive sentences like the medium and no obscure characters.
5. Latency signifcantly increased with each model, can see the run time.


*Vosk Model:*
1. Requires .wav, could potentially be tedius in the business process to have to make the conversion
2. Has no punctuation or start and end times-- so it cannot be considered for buisness purpose. The lack of puntuation is not a big deal because it gets removed in cleaning anyways. But other than that this would have been a viabale candidate for further testing.


*Salad Transcription:*
1. Has Whisper v3 as base model, not open source. Has many different LLM integratiuons which could be appealing for business purposes.
2. Fulfills sentence level time stamps, latency and language requirements.

*Assemly AI:*
1. Similar to salad, not open source, fulfills requirements, less LLM integration opporuntites within one API call.


